# Arrays, memories, vectors

A few array types of the following sort may be required:

* Arrays of signals
* Specific array signals

Array support is currently imported from the `lists` module. It is not part of the kernel.

**Note** Array types can by default not be used in the interface. However, they can be unrolled automatically using procedural interface types, see [UnrollArray](#Procedural-arrays)

In [1]:
from cyhdl import *
from myirl.lists import *

In order to elaborate on the HDL equivalents of the array constructs, we instance a dummy target for direct output:

In [2]:
from myirl.targets.dummy import DummyTargetModule
vl = DummyTargetModule(targets.Verilog)
vh = DummyTargetModule(targets.VHDL)

## Arrays of signals

Arrays of signals are simply defined as follows

In [3]:
from cyhdl import *

a = [ Signal(bool(), name = "a%d" % i) for i in range(18) ]

and are simply used in expressions using indexing:

In [4]:
a[0] + a[1] * a[4]

ADD(a0, MUL(a1, a4))

## Specific Array signals

Arrays of signals **can not** take a variable as index, unlike the special array signal types listed below.

In [5]:
seq = [ intbv()[6:] for _ in range(8) ]
s = SigArray(seq, name='sequence', init=True, elem_size = 7)

In [6]:
for i in s.declare_vhdl(vh, s.identifier):
    vh.output(i)

type a_sequence is array (0 to 7) of unsigned(5 downto 0);
signal sequence : a_sequence := (
    "000000","000000","000000","000000","000000","000000",
    "000000","000000"
);


In [7]:
for i in s.declare_verilog(vl, s.identifier, 'wire'):
    vl.output(i)

wire [5:0] sequence[7:0];
initial begin
	sequence[0] = 6'b000000;
	sequence[1] = 6'b000000;
	sequence[2] = 6'b000000;
	sequence[3] = 6'b000000;
	sequence[4] = 6'b000000;
	sequence[5] = 6'b000000;
	sequence[6] = 6'b000000;
	sequence[7] = 6'b000000;
end


Such signal arrays are declared like signals in the declarative body of the `@block`. For examples, see [RAM and ROM instancing](#RAM-and-ROM-instancing).

## RAM and ROM instancing

For simple, single port ROM or RAM instances, you can use derivations of the above `SigArray` type, like `MemArray`.

See also:

* [myirl.library.memory](../../myirl/library/memory.py) - Memory auxiliaries
* [Cyrite RAM library](cyrite_memories.ipynb)
* Examples: [True dual port RAM](../examples/tdpram.ipynb)

For more complicated RAM types with specific properties, such as true dual port RAMs with various transparency options, it is recommended to use external blackbox libraries for your specific synthesis target.

## Vector signals

Vector signals are basically signal arrays extended by some arithmetic operations.

In [8]:
from myirl.library.vectorsignal import VectorSignal, _VectorSignalAlias

In [9]:
a = VectorSignal(3, intbv()[7:])
b = VectorSignal(3, intbv()[7:])
assert isinstance(a, SigArray)

In [10]:
a.set((3, 2, 1)).evaluate()
b.set((0, 0, 4)).evaluate()

c = VectorSignal(3, intbv()[8:])
op_add = c.set(a + b)
op_add.evaluate()
c.evaluate()

[intbv(3), intbv(2), intbv(5)]

## Procedural arrays

### UnrollSignal and UnrollArray

All the above array type constructs can not be used in the interface as such, or not in a portable way, because the HDL may not support it.

To unroll an array or vector type in the interface into a basic, supported type, an internal auto-casting mechanism is provided to pass vectors as function arguments to a `@block`. During interface construction, they are automatically unrolled into port members.

Likewise, UnrollSignals can be used to create black boxes with single bit I/O requirements, like various FPGA primitives, without the need to list each pin explicitely.

In [11]:
from myirl.library.procedural import UnrollSignal, UnrollArray
from myirl.vector import VectorSig

Since the interface requires some strictness, we define an ad-hoc datatype:

In [12]:
VectorInterface3 = UnrollArray.Type(intbv()[8:], 3)

Then we can define a unit making use of this strict type:

In [13]:
@block
def unit(a : VectorInterface3, q: VectorInterface3.Output):
    v = VectorSig(3, intbv()[8:])
    
    wires = [
        v.wireup(a),
        q.wireup(v)
    ]
    return wires

a, b = [ VectorSig(3, intbv()[8:]) for _ in range(2) ]

u = unit(a, b)
files = u.elab(targets.Verilog)

 Writing 'unit' to file /tmp/myirl_unit_vwnxetc3/unit.v 
DEBUG: Source 'v_8133' is logic: <class 'myirl.vector.VectorSig'>
 Array not used: v_8133 


DEBUG: SKIP NON SIGNAL ARRAY(v_8133)


In [14]:
!cat {files[0]}

// File generated from source:
//     /tmp/ipykernel_84016/1425352769.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture cyriteHDL

module unit
    (
        input wire [7:0] a0,
        input wire [7:0] a1,
        input wire [7:0] a2,
        output wire [7:0] q0,
        output wire [7:0] q1,
        output wire [7:0] q2
    );
    // Local type declarations
    // Signal declarations
    /* unused? */ wire [7:0] v_8133[0:2];
    assign  v_8133[0] = a0; /* fallback */
    assign  v_8133[1] = a1; /* fallback */
    assign  v_8133[2] = a2; /* fallback */
    q0 <= v_8133[0]; /* Unrolled */
    q1 <= v_8133[1]; /* Unrolled */
    q2 <= v_8133[2]; /* Unrolled */
endmodule // unit


## Arrays of values [VHDL]

Arrays of values are supported for VHDL code only.

In [15]:
lut = [ i+5 for i in range(10) ]
v = ValueList(lut, name="values")

A value list will result in a VHDL declaration of the following:

In [16]:
gen = v[0].declare_vhdl(vh, v.identifier)
for i in gen:
    vh.output(i)

type a_values is array (0 to 9) of natural; -- ArrayType
variable values : a_values := (
    5,6,7,8,9,10,11,12,13,14
);
